<a href="https://colab.research.google.com/github/alexmacharia/deep_learning_projects/blob/main/4_PyTorch_Performance_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch

import torchvision
import torchvision.transforms as transforms

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

print("GPU available: {}".format(torch.cuda.is_available()))

device = 'cpu'

GPU available: False
